In [55]:
from collections import Counter
import numpy as np
from pymatgen.io.vasp import Poscar

def types_from_structure(structure):
    elements = Counter(structure.species)
    for e in elements:
        print(e.name)
        print(type(e.name))


In [57]:
structure = Poscar.from_file('poscars/training_set/POSCAR1').structure

In [5]:
types_from_structure(structure)

Li
<class 'str'>
Ni
<class 'str'>
O
<class 'str'>


In [58]:
for site in structure:
    coords=site.coords

In [67]:
np.shape(coords)[0]

3

In [73]:
if np.shape(coords) != (3,):
    print('not right')

In [119]:
potentials = {'Li-O':[10.00, 0.1, 0.0],
              'Ni-O':[100.00, 0.1, 0.0],
              'O-O':[1000.00, 1.0, 1.0]}

In [120]:
potentials

{'Li-O': [10.0, 0.1, 0.0],
 'Ni-O': [100.0, 0.1, 0.0],
 'O-O': [1000.0, 1.0, 1.0]}

In [122]:
for key, pot in potentials.items():
    pot[0]= 1
    potentials[key] = pot 


In [123]:
potentials

{'Li-O': [1, 0.1, 0.0], 'Ni-O': [1, 0.1, 0.0], 'O-O': [1, 1.0, 1.0]}

In [124]:
for key, pot in potentials.items():
    for parameter in pot:
        print(parameter)
        if not isinstance(parameter, float):
            raise TypeError('Potentials values must be of type float.')

1


TypeError: Potentials values must be of type float.